In [1]:
import os
import numpy as np
import math
import random
import datetime

In [2]:
import orekit
vm = orekit.initVM()

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import RandomState
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.attitudes import FrameAlignedProvider
from org.orekit.bodies import CelestialBodyFactory
from org.orekit.bodies import OneAxisEllipsoid
from org.orekit.forces.gravity import HolmesFeatherstoneAttractionModel
from org.orekit.forces.gravity import NewtonianAttraction
from org.orekit.forces.gravity import ThirdBodyAttraction
from org.orekit.forces.gravity.potential import GravityFieldFactory
from org.orekit.forces.maneuvers import ConstantThrustManeuver
from org.orekit.forces.radiation import IsotropicRadiationSingleCoefficient, IsotropicRadiationClassicalConvention
from org.orekit.forces.radiation import SolarRadiationPressure
from org.orekit.frames import FramesFactory
from org.orekit.orbits import KeplerianOrbit
from org.orekit.orbits import Orbit
from org.orekit.orbits import OrbitType
from org.orekit.orbits import PositionAngleType
from org.orekit.propagation import SpacecraftState
from org.orekit.propagation.conversion import DormandPrince853IntegratorBuilder
from org.orekit.propagation.numerical import NumericalPropagator
from org.orekit.time import AbsoluteDate
from org.orekit.time import TimeScalesFactory
from org.orekit.utils import Constants
from org.orekit.utils import IERSConventions
from org.orekit.ssa.metrics import ProbabilityOfCollision
from org.hipparchus.linear import RealMatrix
from org.orekit.propagation import StateCovariance
from org.orekit.frames import FramesFactory
from org.hipparchus.linear import MatrixUtils
from org.orekit.ssa.collision.shorttermencounter.probability.twod import Patera2005

from orekit.pyhelpers import datetime_to_absolutedate

In [4]:
from orekit.pyhelpers import download_orekit_data_curdir, setup_orekit_curdir
download_orekit_data_curdir()
setup_orekit_curdir()

In [5]:
from org.orekit.frames import FramesFactory
gcrf = FramesFactory.getGCRF()

from org.orekit.time import TimeScalesFactory
utc = TimeScalesFactory.getUTC()

In [6]:
# keplerian elements of ISS
sma = 6795.e3
ecc = 0.00048
inc = 51.6413  # deg
argp = 21.0174  # deg
raan = 60  # deg
tran = 0.0

ref_mass = 100.0  # kg
spacecraft_area = 1.0 # m^2
spacecraft_reflection = 2.0 # Perfect reflection

In [7]:
def deg_to_rad(deg: float)-> float:
    return (math.pi * deg) / 180.0

def get_orbital_period(sma: float):
    return 2.0 * np.pi * np.sqrt(np.divide(np.power(sma, 3), Constants.WGS84_EARTH_MU))

In [8]:
orb_period = get_orbital_period(sma=sma)
orb_period/60

92.90603411371595

In [9]:
inc_rad = deg_to_rad(inc)
argp_rad = deg_to_rad(argp)
raan_rad = deg_to_rad(raan)

In [10]:
# initial SV
ref_sv = np.array([sma, ecc, inc_rad, argp_rad, raan_rad, tran])
ref_frame = gcrf
ref_sc_frame = gcrf
ref_time = AbsoluteDate(2023, 6, 16, 0, 0, 0.0, TimeScalesFactory.getUTC())

In [11]:
# create the kepl orbit object
kep = ref_sv.tolist()
orbit1 = KeplerianOrbit(kep[0], kep[1], kep[2], kep[3], kep[4], kep[5],
                        PositionAngleType.MEAN, ref_frame, ref_time, Constants.WGS84_EARTH_MU)
orbit2 = KeplerianOrbit(kep[0] - 5, kep[1], kep[2], kep[3], kep[4], kep[5],
                        PositionAngleType.MEAN, ref_frame, ref_time, Constants.WGS84_EARTH_MU)

spacecraft_state = SpacecraftState(orbit1, ref_mass)

In [17]:
def create_propagator(orbit: Orbit, earth_order: float, earth_degree: float, use_perturbations: bool = True):    
    # create the propagator
    orbit_type = orbit.getType()
    integrator = DormandPrince853IntegratorBuilder(1.0, 1000., 1.0).buildIntegrator(orbit, orbit_type)

    propagator = NumericalPropagator(integrator)
    propagator.setOrbitType(orbit_type)
    propagator.setInitialState(spacecraft_state)

    # Earth gravity field
    if not use_perturbations:
        point_gravity = NewtonianAttraction(Constants.WGS84_EARTH_MU)
        propagator.addForceModel(point_gravity)
    else:
        earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS,
                                Constants.WGS84_EARTH_FLATTENING,
                                FramesFactory.getITRF(IERSConventions.IERS_2010, True))
        harmonics_gravity_provider = GravityFieldFactory.getNormalizedProvider(earth_degree, earth_order)
        propagator.addForceModel(
            HolmesFeatherstoneAttractionModel(earth.getBodyFrame(), harmonics_gravity_provider))

        # Sun and Moon attraction
        propagator.addForceModel(ThirdBodyAttraction(CelestialBodyFactory.getSun()))
        propagator.addForceModel(ThirdBodyAttraction(CelestialBodyFactory.getMoon()))

        # Solar radiation pressure
        propagator.addForceModel(
            SolarRadiationPressure(CelestialBodyFactory.getSun(),
                                earth,
                                IsotropicRadiationSingleCoefficient(spacecraft_area,
                                                                    spacecraft_reflection)))

    rotation = FramesFactory.getEME2000().getTransformTo(ref_sc_frame, ref_time).getRotation()
    attitude = FrameAlignedProvider(rotation)
    propagator.setAttitudeProvider(attitude)

    return propagator

In [18]:
propagator = create_propagator(orbit=orbit1, earth_order=16, earth_degree=16, use_perturbations=True)

In [19]:
propagator

<NumericalPropagator: org.orekit.propagation.numerical.NumericalPropagator@7123be6c>

In [ ]:
def generate_symmetric_covariance_matrix(size):
    # Create an empty matrix of the desired size
    matrix = MatrixUtils.createRealMatrix(size, size)
    
    # Fill the upper triangle and diagonal with random values
    for i in range(size):
        for j in range(i, size):
            value = random.random()
            matrix.setEntry(i, j, value)
            matrix.setEntry(j, i, value)  # Mirror to make it symmetric
    
    return matrix

In [ ]:
# set tca
tca = datetime.datetime(2023, 6, 16, 0, 0, 0)

In [ ]:
# covariance_values3 = [
#     [-5.826e-1, 2.538e-2, -2.476e-6, -1.628e-4, -1.782e-4, 1.605e-4],
#     [2.538e-2, 7.537e-1, -8.935e-2, -2.343e-4, -7.149e-4, 5.660e-4],
#     [-2.476e-6, -8.935e-2, 9.269e-1, 2.591e-4, 6.95e-4, -7.503e-4],
#     [-1.628e-4, -2.343e-4, 2.591e-4,2.591e-7, 4.042e-7, -3.707e-7],
#     [-1.782e-4, -7.149e-4, 6.95e-4, 4.042e-7, 1.198e-6, -9.648e-7],
#     [1.605e-4, 5.660e-4, -7.503e-4, -3.707e-7, -9.648e-7, 1.066e-6]]

covariance_values3 = [
    [-9.826e-4, 2.538e-2, -2.476e-6, -1.628e-4, -1.782e-4, 1.605e-4],
    [2.538e-2, 7.537e-5, -8.935e-2, -2.343e-4, -7.149e-4, 5.660e-4],
    [-2.476e-1, -8.935e-2, 9.269e-5, 2.591e-4, 6.95e-4, -7.503e-4],
    [-1.628e-1, -2.343e-4, 2.591e-4,2.591e-5, 4.042e-7, -3.707e-7],
    [-1.782e-1, -7.149e-4, 6.95e-4, 4.042e-1, 1.198e-5, -9.648e-7],
    [1.605e-1, 5.660e-4, -7.503e-1, -3.707e-1, -9.648e-1, 1.066e-1]]

# Convert the Python list to a Java double[][]
jarray = MatrixUtils.createRealMatrix(len(covariance_values3), len(covariance_values3[0]))
for i in range(len(covariance_values3)):
    for j in range(len(covariance_values3[i])):
        try:
            jarray.setEntry(i, j, covariance_values3[i][j])
        except:
            print(i, j, covariance_values3[i][j])
            print(f"value: {covariance_values3[i][j]}")
            raise

In [ ]:
# Use MatrixUtils to create a RealMatrix object from the 2D array
cov_mat1 = jarray
cov_mat2  = jarray

def symmetrize(matrix):
    nrows, ncols = matrix.getRowDimension(), matrix.getColumnDimension()
    for i in range(nrows):
        for j in range(i+1, ncols):
            value = (matrix.getEntry(i, j) + matrix.getEntry(j, i)) / 2.0
            matrix.setEntry(i, j, value)
            matrix.setEntry(j, i, value)

# Apply symmetrization covariance
symmetrize(cov_mat1)
symmetrize(cov_mat2)

In [ ]:
# create covariance matrices
cov_mat1 = generate_symmetric_covariance_matrix(6)
cov_mat2 = generate_symmetric_covariance_matrix(6)

In [ ]:
# compute probability of collision
radius1 = 1.0
radius2 = 2.0

covariance1 = StateCovariance(cov_mat1, datetime_to_absolutedate(tca), gcrf, OrbitType.CARTESIAN, PositionAngleType.TRUE)
covariance2 = StateCovariance(cov_mat2, datetime_to_absolutedate(tca), gcrf, OrbitType.CARTESIAN, PositionAngleType.TRUE)

# Patera2005.compute(Orbit primaryAtTCA, StateCovariance primaryCovariance, double primaryRadius, Orbit secondaryAtTCA, StateCovariance secondaryCovariance, double secondaryRadius)
patera2005 = Patera2005() 
poc_result = patera2005.compute(orbit1, covariance1, orbit2, covariance2, radius2, 1e-10)
print(f"Probability of collision: {poc_result.getValue()}")

In [ ]:
class PerigeeRaisingEnv:
    def __init__(self, use_perturbations=False, perturb_action=False, **kwargs):

        self._ref_time = AbsoluteDate(2022, 6, 16, 0, 0, 0.0, TimeScalesFactory.getUTC())
        self._ref_frame = FramesFactory.getGCRF()
        self._ref_sv = np.array([sma, ecc, inc, argp, raan, tran])
        self._ref_sv_pert = np.array([0.0, 0.0, 0.0, 0.0, 0.0, math.pi])
        self._ref_mass = 100.0  # Kg
        self._ref_sc_frame = FramesFactory.getGCRF()
        self._use_perturbations = use_perturbations
        self._perturb_action = perturb_action
        self._earth_degree = 16
        self._earth_order = 16

        self._spacecraft_area = 1.0  # m^2
        self._spacecraft_reflection = 2.0  # Perfect reflection
        self._thruster_max_force = 0.01  # N
        self._thruster_isp = 4000.0  # s

        self._time_step = 60.0 * 5.0  # 5 minutes
        self._max_steps = 166

        min_pos = self._ref_sv[0] * (1.0 - self._ref_sv[1])
        max_pos = self._ref_sv[0] * (1.0 + self._ref_sv[1])
        max_vel = np.sqrt(Constants.WGS84_EARTH_MU * (2.0 / min_pos - 1.0 / self._ref_sv[0]))
        box = np.array([self._time_step * self._max_steps * 1.1,
                        max_pos * 1.1, max_pos * 1.1, max_pos * 1.1,
                        max_vel * 1.1, max_vel * 1.1, max_vel * 1.1,
                        self._ref_mass * 1.1])

        self._propagator = None
        self.hist_sc_state = None
        self.hist_action = None
        self.prev_hist_sc_state = None
        self.prev_hist_action = None
        self._current_step = None
        self._random_generator = None

        self.close()
        self.seed()
        self.reset()

In [ ]:
def _create_propagator(self):
    kep = (self._ref_sv + (self._random_generator.rand(6) * 2. - 1.) * self._ref_sv_pert).tolist()
    orbit = KeplerianOrbit(kep[0], kep[1], kep[2], kep[3], kep[4], kep[5],
                            PositionAngleType.MEAN, self._ref_frame, self._ref_time, Constants.WGS84_EARTH_MU)

    integrator = DormandPrince853IntegratorBuilder(1.0, 1000., 1.0)\
        .buildIntegrator(orbit, OrbitType.CARTESIAN)
    propagator = NumericalPropagator(integrator)
    propagator.setSlaveMode()
    propagator.setOrbitType(OrbitType.CARTESIAN)
    propagator.setInitialState(SpacecraftState(orbit, self._ref_mass))

    # Earth gravity field
    if not self._use_perturbations:
        point_gravity = NewtonianAttraction(Constants.WGS84_EARTH_MU)
        propagator.addForceModel(point_gravity)
    else:
        earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS,
                                    Constants.WGS84_EARTH_FLATTENING,
                                    FramesFactory.getITRF(IERSConventions.IERS_2010, True))
        harmonics_gravity_provider = GravityFieldFactory.getNormalizedProvider(self._earth_degree, self._earth_order)
        propagator.addForceModel(
            HolmesFeatherstoneAttractionModel(earth.getBodyFrame(), harmonics_gravity_provider))

    if self._use_perturbations:
        # Sun and Moon attraction
        propagator.addForceModel(ThirdBodyAttraction(CelestialBodyFactory.getSun()))
        propagator.addForceModel(ThirdBodyAttraction(CelestialBodyFactory.getMoon()))

        # Solar radiation pressure
        propagator.addForceModel(
            SolarRadiationPressure(CelestialBodyFactory.getSun(),
                                    earth.getEquatorialRadius(),
                                    IsotropicRadiationSingleCoefficient(self._spacecraft_area,
                                                                        self._spacecraft_reflection)))

    rotation = FramesFactory.getEME2000().getTransformTo(self._ref_sc_frame, self._ref_time).getRotation()
    attitude = FrameAlignedProvider(rotation)
    propagator.setAttitudeProvider(attitude)
    return propagator